#### Peter Dresslar, OMT 548, Summer C 2025

This example is from a real-life chore I needed to complete recently. I had two spreadsheets, old and new, (converted to csv files) and needed to integrate the data from both into one sheet in tricky ways:

- The old sheet has no ID row; the new one does---called "ITEM NUMBER"
- There is no single key in the old sheet but we can set a compound key
- The new sheet has additional (new rows)
- The new sheet is missing some expense mappings from the old sheet that were manually-created and expensive to make. We need to preserve old sheet mappings. This is in column "Budget Line Item."

The following example is a "pure Julia" approach, although note that there is a bit of a pipeline. The comments are verbose.



In [56]:
using CSV
using DataFrames
using Statistics

# Read the CSV files directly into DataFrames
old_df = DataFrame(CSV.File("1.csv"))

# drop the last column of old_df, it is empty
old_df = old_df[:, 1:end-1]

new_df = DataFrame(CSV.File("2.csv"))

# clean old_df column names to lowercase, underscores for spaces
# while we are at it, make sure the budget_line_item is a string
function cleancols!(df)
    for col in names(df)
       new_name = lowercase(replace(string(col), " " => "_"))
       rename!(df, col => new_name)
       if new_name == "budget_line_item"
           df.budget_line_item = string.(df.budget_line_item)
       end
    end
end

cleancols!(old_df)  # not strictly necessary, but feels better
cleancols!(new_df)

# println(old_df)
# println(new_df)

println("old rows ", nrow(old_df))
println("new rows ", nrow(new_df))


# see https://dataframes.juliadata.org/stable/man/split_apply_combine/
old_df_unique = combine(   # compound key using combine()
    groupby(old_df, ["document", "description", "amount"], skipmissing=false),
    "budget_line_item" => first => "old_budget_line_item" # pipeline to apply the old budget line items to the old budget rows
)

println("compound keys in old sheet ", nrow(old_df_unique))

# println(old_df_unique)

# Perform the merge: leftjoin to keep all new rows
merged_df = leftjoin(new_df, old_df_unique, 
    on = ["document", "description", "amount"],
    matchmissing = :equal)  # matchmissing = :equal or otherwise new rows are dropped.

println("merged rows ", nrow(merged_df))

# note that since we have new rows we will still have some empty "-" budget_line_items
merged_df.budget_line_item = ifelse.(
    # if the budget_line_item is missing or "-"
    ismissing.(merged_df.budget_line_item) .| (merged_df.budget_line_item .== "-"),
    # then use the old_budget_line_item
    coalesce.(merged_df.old_budget_line_item, merged_df.budget_line_item),
    # otherwise use the budget_line_item
    merged_df.budget_line_item
)

# drop the temporary column
select!(merged_df, Not("old_budget_line_item"))

# Sort by item_number
sorted_df = sort(merged_df, "item_number")

println("final row count: ", nrow(sorted_df))

println(sorted_df)

#output the sorted_df to a csv file
CSV.write("sorted_df1.csv", sorted_df)


old rows 8
new rows 11
compound keys in old sheet 8
merged rows 11
final row count: 11
11×8 DataFrame
 Row │ item_number  document  budget_line_item  document_date  label            description             amount       account 
     │ Int64        Int64     AbstractString    String15       String15?        String31                String15     Int64   
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        1023         1  45                5/26/2023      PO20607          ABC LLC                 $9,900.00      -5505
   2 │        1122         7  9                 5/30/2024      5/11/24          Retirement              -              -5123
   3 │        1123         7  9                 5/30/2024      5/11/24          Workmen's Compensation  $100.93        -5124
   4 │        1127         7  76                5/30/2024      missing          INDIRECT COST           $3,656.98      -5310
   5 │        1192  

"sorted_df1.csv"

Following is a version of the exact same chore using Tidier.jl:

https://tidierorg.github.io/TidierData.jl/latest/

From that website:

> TidierData.jl is a 100% Julia implementation of the dplyr and tidyr R packages. Powered by the DataFrames.jl package and Julia’s extensive meta-programming capabilities, TidierData.jl is an R user’s love letter to data analysis in Julia.

Let's check it out.


In [57]:
using TidierData

# import the data into the new dataframes, which are just jl.DataFrames
old_df_tidier = DataFrame(CSV.File("1.csv"))
new_df_tidier = DataFrame(CSV.File("2.csv"))

# drop the last column of old_df_tidier, it is empty
old_df_tidier = old_df_tidier[:, 1:end-1]  # there is not really a tidier shorthand

function cleancols_tidier(df)
    @chain df begin
        @clean_names() # nice!!! defaults to lowercase and underscores
        @mutate(budget_line_item = string(budget_line_item))
    end
end

# clean the column names

old_df_tidier = cleancols_tidier(old_df_tidier)
new_df_tidier = cleancols_tidier(new_df_tidier)

# println(old_df_tidier)
# println(new_df_tidier)

println("old rows ", nrow(old_df))
println("new rows ", nrow(new_df))

# get the compound keys and old budget line items
old_df_tidier_unique = @chain old_df_tidier begin
    @group_by(document, description, amount)
    @summarize(old_budget_line_item = first(budget_line_item))
    @ungroup()
end

println("compound keys in old sheet ", nrow(old_df_tidier_unique))

# println(old_df_tidier_unique)

# now we are ready to join using tidier
# tidier has a natural join capability we can use for this
merged_df_tidier = @left_join(new_df_tidier, old_df_tidier_unique)

# println(merged_df_tidier)

println("merged rows ", nrow(merged_df_tidier))

# now we need to map the old_budget_line_item to the budget_line_item where needed
# while we are at it we can finish up our tidy pipeline
merged_df_tidier = @chain merged_df_tidier begin
    @mutate(budget_line_item = ifelse.(
        # if the budget_line_item is missing or "-"
        ismissing.(budget_line_item) .| (budget_line_item .== "-"),
        # then use the old_budget_line_item
        coalesce.(old_budget_line_item, budget_line_item),
        # otherwise use the budget_line_item
        budget_line_item
    ))
    @select(Not(:old_budget_line_item)) # duump the column now that we have used it
    @arrange(item_number) # sort by item_number
end

println(merged_df_tidier)

# output the merged_df_tidier to a csv file
CSV.write("sorted_df_tidier.csv", merged_df_tidier)


old rows 8
new rows 11
compound keys in old sheet 8
merged rows 11
11×8 DataFrame
 Row │ item_number  document  budget_line_item  document_date  label            description             amount       account 
     │ Int64        Int64     AbstractString    String15       String15?        String31                String15     Int64   
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │        1023         1  45                5/26/2023      PO20607          ABC LLC                 $9,900.00      -5505
   2 │        1122         7  9                 5/30/2024      5/11/24          Retirement              -              -5123
   3 │        1123         7  9                 5/30/2024      5/11/24          Workmen's Compensation  $100.93        -5124
   4 │        1127         7  76                5/30/2024      missing          INDIRECT COST           $3,656.98      -5310
   5 │        1192         7  61        

"sorted_df_tidier.csv"